In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd

In [8]:
df = pd.read_csv('data/used_cars.csv')
print(list(df.columns))

# data cleaning
df = df.dropna()
df['fuel_type'].unique()
df_new = df[df['fuel_type'] != "not supported"]
df_new['fuel_type'].unique()
df_new = df_new[df_new['fuel_type'] != '–']

df_new.drop(columns=["model_year", "milage","engine","transmission","ext_col","int_col","accident","clean_title"], inplace=True)

# Remove the dollar sign and convert to numeric
df_new['price'] = df_new['price'].str.replace(r'[$,]', '', regex=True).astype(float)
print(df_new.head())

['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title', 'price']
      brand                            model      fuel_type    price
0      Ford  Utility Police Interceptor Base  E85 Flex Fuel  10300.0
1   Hyundai                     Palisade SEL       Gasoline  38005.0
3  INFINITI                 Q50 Hybrid Sport         Hybrid  15500.0
6      Audi             S3 2.0T Premium Plus       Gasoline  31000.0
7       BMW                           740 iL       Gasoline   7300.0


In [10]:
df_new['fuel_type'].unique()
df_new["CarbonTax"] = [
    5 if type == "Diesel" else
    4 if type == "Gasoline" else
    3 if type == 'E85 Flex Fuel' else
    2 if type == "Hybrid" else
    1
    for type in df_new["fuel_type"]
]
print(df_new.head())

      brand                            model      fuel_type    price  \
0      Ford  Utility Police Interceptor Base  E85 Flex Fuel  10300.0   
1   Hyundai                     Palisade SEL       Gasoline  38005.0   
3  INFINITI                 Q50 Hybrid Sport         Hybrid  15500.0   
6      Audi             S3 2.0T Premium Plus       Gasoline  31000.0   
7       BMW                           740 iL       Gasoline   7300.0   

   CarbonTax  
0          3  
1          4  
3          2  
6          4  
7          4  


In [18]:
# we have ten cars waiting to be chosen
N = len(df_new)
print(N)
# value of cars varies
v = df_new['price']


# size of the cars varies
s = np.random.randint(8,10,N)
# total size of all cars
size = np.sum(s)
# parking lot capacity
S = np.random.randint(size//2,2*size//3)

# carbon tax for each car
c = df_new["CarbonTax"]
# total amount of carbon tax
tax = np.sum(c)
# total carbon tax the dealership would pay
C = np.random.randint(tax//2,2*tax//3)

print("Total value =",np.sum(v))
print("Total Size =",size)
print("Total carbon tax amount = ", tax)
print("------------------------------------------")
print("Total Capacity =",S)
print("Total Carbon tax paid = $", C)

# Solution
x = cp.Variable(N,integer=True)
obj = cp.Maximize(cp.sum(cp.multiply(v,x)))
constraints = [cp.sum(cp.multiply(s,x)) <= S,
               cp.sum(cp.multiply(c,x)) <= C,
                 x <= 1, x >= 0]
prob = cp.Problem(obj,constraints)
prob.solve()


# counter = 1
# for i in x.value:
#     if i==1:
#       print(counter)
#     counter+=1

print(np.sum(x.value), "cars sold")
print("Profit: $", v@x.value)
print("Actual Size of cars sold: ", s@x.value)
print("Actual carbon tax spent: $", c@x.value)

3223
Total value = 133255677.0
Total Size = 27409
Total carbon tax amount =  12477
------------------------------------------
Total Capacity = 14222
Total Carbon tax paid = $ 6875
1677.0 cars sold
Profit: $ 109669524.0
Actual Size of cars sold:  14222.0
Actual carbon tax spent: $ 6470.0


In [12]:
print(sum(x.value))
N

1818.0


3223